# Binary Classification with a Tabular Stroke Prediction Dataset

**According to the World Health Organization (WHO) stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths. This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.**

1. id: unique identifier
2. gender: "Male", "Female" or "Other"
3. age: age of the patient
4. hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5. heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6. ever_married: "No" or "Yes"
7. work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8. Residence_type: "Rural" or "Urban"
9. avg_glucose_level: average glucose level in blood
10. bmi: body mass index
11. smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
12. stroke: 1 if the patient had a stroke or 0 if not

***Note: "Unknown" in smoking_status means that the information is unavailable for this patient**

# IMPORT THE NECESSARY LIBRARIES

In [1]:
import numpy as np # Linear Algrebra
import pandas as pd # Data Processing

import os
import random
from collections import defaultdict
from statistics import mean
from bisect import bisect_left

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor # Import KNN Regressor
from xgboost import XGBClassifier # Import xgboost

# LOAD THE DATA

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

# load the external dataset
ext_data = pd.read_csv('healthcare-dataset-stroke-data.csv')
ext_data = ext_data[ext_data['stroke'] == 1]

# FEATURE ENGINEERING:

In [3]:
FEATURES = train.columns.to_list()[1:11]
FEATURES.append('risk_factors')
CAT_FEATURES = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]
CON_FEATURES = ['age', 'avg_glucose_level', 'bmi']
TARGET = 'stroke'

# ONE HOT ENCODING:

In [4]:
# one hot encoding of categorical features
train_ohe = train.copy()
test_ohe = test.copy()
ext_ohe = ext_data.copy()

for feature in CAT_FEATURES:
    ohe = OneHotEncoder()
    train_temp = pd.DataFrame(train_ohe[feature])
    test_temp = pd.DataFrame(test_ohe[feature])
    ext_temp = pd.DataFrame(ext_ohe[feature])
    
    merged_temp = pd.DataFrame(train_ohe[feature])
    merged_temp = merged_temp.append(pd.DataFrame(test_ohe[feature]), ignore_index = True)
    merged_temp = merged_temp.append(pd.DataFrame(ext_ohe[feature]), ignore_index = True)
   
    ohe = OneHotEncoder(sparse = False, drop = 'first')
    ohe.fit(merged_temp)
    
    new_columns = ["{}_{}_ohe".format(feature, val) for val in ohe.categories_[0][1:]]
    
    train_ohe_column = pd.DataFrame(ohe.transform(train_temp), columns = new_columns)
    test_ohe_column = pd.DataFrame(ohe.transform(test_temp), columns = new_columns)
    ext_ohe_column = pd.DataFrame(ohe.transform(ext_temp), columns = new_columns)
    
    for column in new_columns:
        train_ohe[column] = train_ohe_column[column]
        test_ohe[column] = test_ohe_column[column]
        ext_ohe[column] = ext_ohe_column[column]
        FEATURES.append(column)
    
for feature in ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]:
    FEATURES.remove(feature)
    
train_ohe.drop(CAT_FEATURES, axis = 1, inplace = True)
train_ohe.drop("id", axis = 1, inplace = True)

test_ohe.drop(CAT_FEATURES, axis = 1, inplace = True)
test_ohe.drop("id", axis = 1, inplace = True)

ext_ohe.drop(CAT_FEATURES, axis = 1, inplace = True)
ext_ohe.drop('id', axis = 1, inplace = True)

C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\3300590819.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_temp = merged_temp.append(pd.DataFrame(test_ohe[feature]), ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\3300590819.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_temp = merged_temp.append(pd.DataFrame(ext_ohe[feature]), ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\3300590819.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_temp = merged_temp.append(pd.DataFrame(test_ohe[feature]), ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\3300590819.py:14: FutureWarning: The frame.append method is depre

# Imputing BMI
**In adding the stroke samples from the original dataset, some observations have no entry for BMI and needed imputation.**

In [5]:
# manual evaluation of best estimates of BMI showed that age by 
# itself gave nearly the best performance.  avg_glucose_level 
# added about 1% improvement.  All others did not help and some
# made the estimate worse.

KNNCOLS = [
            'age', 
#             'gender_Male_ohe', 'gender_Other_ohe',
#             'hypertension', 
#             'heart_disease', 
            'avg_glucose_level', 
#             'work_type_Never_worked_ohe', 'work_type_Private_ohe', 'work_type_Self-employed_ohe', 'work_type_children_ohe',
#             'Residence_type_Urban_ohe', 
#             'smoking_status_formerly smoked_ohe', 'smoking_status_never smoked_ohe', 'smoking_status_smokes_ohe'
]

ext_impute = ext_ohe.copy()

knn = KNeighborsRegressor(n_neighbors = 100,
                          metric = 'minkowski',
                          n_jobs = -1)
knn.fit(ext_impute[KNNCOLS], ext_impute[TARGET])
dists, nears = knn.kneighbors(ext_impute[KNNCOLS], return_distance = True)

# calculate error on predictions
result = []
for i, n in enumerate(nears):
    n = list(n)
    
    # depending on the features used, the ith index may not be
    # in the nearest neihbors list
    # for categorical features only, sometimes all distances are zero.
    # using just the median of the whole dataset gives RMSE 6.26
    try:
        n.remove(i)
    except:
        continue

    try:
        avg_bmi = ext_impute.iloc[n]['bmi'].median()
        if( not pd.isna(ext_impute.iloc[i]['bmi']) ):
            result.append( (ext_impute.iloc[i]['bmi'] - avg_bmi)**2 )
    except:
        continue

print(f'RMSE: {round(np.mean(result) ** 0.5, 2)}')
        
result = []
for i in ext_impute.query('bmi!=bmi').index:
    result.append(round(ext_impute.iloc[nears[i]]['bmi'].median(),1))
ext_impute.loc[ext_impute.query('bmi!=bmi').index, 'bmi'] = result
ext_ohe['bmi'] = ext_impute['bmi']

RMSE: 6.06


# Computing total risk factors

In [6]:
def feature_risk_factors(df):
    df["risk_factors"] = df[[
        "avg_glucose_level", "age", "bmi", 
        "hypertension", "heart_disease", 
        "smoking_status"
    ]].apply(
        lambda x: \
        0 + (1 if x.avg_glucose_level > 116.5 else 0) + \
        (1 if x.age > 52.5 else 0) + (1 if x.bmi > 25.5 else 0) + \
        (1 if x.hypertension == 1 else 0) + \
        (1 if x.heart_disease == 1 else 0) + \
        (1 if x.smoking_status in ["formerly smoked", "smokes"] else 0),
        axis=1
    )
    return df

train_ohe['risk_factors'] = feature_risk_factors(train)['risk_factors']
test_ohe['risk_factors'] = feature_risk_factors(test)['risk_factors']
ext_ohe['risk_factors'] = feature_risk_factors(ext_data)['risk_factors']

# Center and Scale:

In [7]:
for feature in CON_FEATURES:
    mu = np.mean(train_ohe[feature])
    sigma = np.std(train_ohe[feature])
    train_ohe[feature] = (train_ohe[feature] - mu) / sigma
    test_ohe[feature] = (test_ohe[feature] - mu) / sigma
    ext_ohe[feature] = (ext_ohe[feature] - mu) / sigma

# XgBoost Model:

### Tuning

**The overall premise is quite simple:**

* Do random search across various hyperparameters for XGBoost for 500 iterations.
* Each iteration is run with 10-fold cross-validation.  Data from the original dataset are added to the training set for each cross-validation fold, but **only the synthetic dataset is used for CV calculations.**  Addition of the extra data gave me a CV and LB boost consistently, and only monitoring CV on the synthetic set helped avoid over-training to the original set.
* At each CV fold, a prediction is made on the test set.
* Average CV across 10-folds is monitored and the models are orderd by 10-fold CV average AUC.
* Predictiosn for the test set are made at each fold.
* Models with the highest AUCs are used to create an ensemble predictions later.

In [8]:
# define tuning parameters
n_estimators_values = [10, 25, 50, 100, 150, 200, 250, 300]
eta_values = [ v / 10 for v in range(10) ]
max_depth_values = [2, 4, 6, 8, 10]
subsample_values = [0.25, 0.50, 0.75, 0.90]
colsample_bytree_values = [0.25, 0.50, 0.75, 0.90]

cv_folds = 10
tuning_steps = 500
include_orig = True
tuning_results = defaultdict(list)

col_names = [f'XGB_Step_{step}_Fold_{fold}' 
                 for step in range(tuning_steps) 
                     for fold in range(cv_folds)]
test_predictions = pd.DataFrame(0, index = test_ohe.index, columns = col_names)
valid_predictions = pd.DataFrame(0, index = train_ohe.index, columns = col_names)

random.seed(2201014)

skf_seed = random.randint(0, 2023)
skf = StratifiedKFold(n_splits = cv_folds, random_state = skf_seed, shuffle = True)

for step in range(tuning_steps):
    n_estimators = random.choice(n_estimators_values)
    eta = random.choice(eta_values)
    max_depth = random.choice(max_depth_values)
    subsample = random.choice(subsample_values)
    colsample_bytree = random.choice(colsample_bytree_values)
    
    aucs = []
    test_probs = []

    for i, (train_index, val_index) in enumerate(skf.split(train_ohe[FEATURES], train_ohe[TARGET])):
        X_train, X_val = train_ohe[FEATURES].iloc[train_index], train_ohe[FEATURES].iloc[val_index]
        y_train, y_val = train_ohe[TARGET].iloc[train_index], train_ohe[TARGET].iloc[val_index]

        if include_orig:
            # add in original stroke samples
            X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
            y_train = y_train.append(ext_ohe[TARGET], ignore_index = True)
        
        xgb_seed = random.randint(0, 2023)
        xgb = XGBClassifier(n_estimators = n_estimators,
                            eta = eta,
                            max_depth = max_depth,
                            subsample = subsample,
                            colsample_bytree = colsample_bytree,
                            random_state = xgb_seed).fit(X_train.values, y_train)
        
        val_probs = [probs[1] for probs in xgb.predict_proba(X_val[FEATURES])]
        valid_predictions.loc[val_index, f'XGB_Step_{step}_Fold_{i}'] = val_probs
        
        fpr, tpr, thresholds = metrics.roc_curve(y_val, val_probs, pos_label = 1)
        auc = metrics.auc(fpr, tpr)
        aucs.append(auc)
        
        test_predictions[f'XGB_Step_{step}_Fold_{i}'] = \
            [probs[1] for probs in xgb.predict_proba(test_ohe[FEATURES])]
    
    tuning_results['step'].append(step)
    tuning_results['auc'].append(mean(aucs))
    tuning_results['n_estimators'].append(n_estimators)
    tuning_results['eta'].append(eta)
    tuning_results['max_depth'].append(max_depth)
    tuning_results['subsample'].append(subsample)
    tuning_results['colsample_bytree'].append(colsample_bytree)
    tuning_results['skf_seed'].append(skf_seed)
    tuning_results['xgb_seed'].append(xgb_seed)
    
    print(f'Step: {step}  AUC: {mean(aucs)}')
    
valid_predictions.to_csv('XGBoost_valid_predictions.csv', index = False)
test_predictions.to_csv('XGBoost_test_predictions.csv', index = False)

tuning_results = pd.DataFrame(tuning_results)
tuning_results.sort_values(by = 'auc', axis = 0, inplace = True, ascending = False)
tuning_results.to_csv('XGBoost_tuning_results.csv', index = False)
tuning_results

C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(ext_ohe[TARGET], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pa

Step: 0  AUC: 0.8412074412005369


C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(ext_ohe[TARGET], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pa

Step: 1  AUC: 0.8692558188992516


C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(ext_ohe[TARGET], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pa

...

C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(ext_ohe[TARGET], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pa

Step: 498  AUC: 0.5


C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(ext_ohe[TARGET], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(ext_ohe[FEATURES], ignore_index = True)
C:\Users\2211592\AppData\Local\Temp\1\ipykernel_26464\2793029690.py:41: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pa

Step: 499  AUC: 0.8347651162553382


,step,auc,n_estimators,eta,max_depth,subsample,colsample_bytree,skf_seed,xgb_seed
366,366,0.891278,300,0.2,2,0.90,0.25,1935,132
54,54,0.890995,200,0.2,2,0.75,0.50,1935,1236
325,325,0.890878,100,0.2,2,0.90,0.50,1935,1569
294,294,0.890742,250,0.1,2,0.90,0.75,1935,311
321,321,0.890742,50,0.2,4,0.50,0.50,1935,70
...,...,...,...,...,...,...,...,...,...
403,403,0.500000,150,0.0,10,0.50,0.75,1935,506
151,151,0.500000,50,0.0,8,0.25,0.75,1935,976
189,189,0.500000,25,0.0,2,0.75,0.90,1935,141
296,296,0.500000,25,0.0,4,0.75,0.25,1935,1522


# Creating Submission:

**Ensemble of top 5 model CV ensembles. The final entry was developed from the average of the predictions across the 10-folds for the best 5 models - 50 predictions for the test set were averaged.**

In [9]:
sub = sub.copy()

best_cols = [f'XGB_Step_{step}_Fold_{fold}' 
               for step in tuning_results['step'][0:5]
                   for fold in range(cv_folds)]
cv_probs = test_predictions[best_cols].mean(axis = 1).round(decimals = 4)

sub['stroke'] = cv_probs
sub.to_csv('submisson.csv', index = False)